<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#housekeeping" data-toc-modified-id="housekeeping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>housekeeping</a></span><ul class="toc-item"><li><span><a href="#languages-&amp;-packages" data-toc-modified-id="languages-&amp;-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>languages &amp; packages</a></span></li><li><span><a href="#env-set-up" data-toc-modified-id="env-set-up-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>env set-up</a></span></li><li><span><a href="#model-function" data-toc-modified-id="model-function-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>model function</a></span></li></ul></li><li><span><a href="#step-by-step-numerical-solve-via-SymPy" data-toc-modified-id="step-by-step-numerical-solve-via-SymPy-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>step-by-step numerical solve via SymPy</a></span></li></ul></div>

# housekeeping
This notebook is for analytically solving parameters in the set of ODEs for eco-bioelectric cell (EBC).

## languages & packages
Julia-lang 1.3.1: `PyCall` (1.91.2), `DataFrames` (0.20.0), `SymPy` (1.0.18)

## env set-up

In [19]:
using PyCall, DataFrames, SymPy
mt = pyimport("math")
sc = pyimport("scipy")
itg = pyimport("scipy.integrate")
println()

UndefVarError: UndefVarError: init_printing not defined

## model function
![](graph/PBD_00.png)

In [4]:
function ebc0(popDen, t, gp,ep,Rp,mp, gc,ec,Rc,mc)
    
    ## population densities
    P = popDen[:1]
    M = popDen[:2]
    C = popDen[:3]
    
    ## numerical adaptation to net growth
    #if gp <1; gp = 1+gp;end
    #if gc <1; gc = 1+gc;end
    
    ## fluctuation model
    dp = P*(gp*ep - Rp - mp)
    dm = P*(gp*(1-ep) + mp) + C*(gc*(1-ec)*M + mc)
    dc = C*(gc*ec*M - Rc - mc)
    
    ## non-recoverable elimination
    if P <=0; dp=0;end
    if C <=0; dc=0;end
    
    return(sc.array([dp,dm,dc]))
end

ebc0 (generic function with 1 method)

# step-by-step numerical solve via SymPy
This step-wise workflow is following CMEE [notes](https://nbviewer.jupyter.org/github/mhasoba/TheMulQuaBio/blob/master/notebooks/Appendix-Maths.ipynb) (last section) from Samraat on Jupyter math models  
define variables

In [15]:
@vars P; @vars M; @vars C
@vars gp; @vars ep; @vars Rp; @vars mp
@vars gc; @vars ec; @vars Rc; @vars mc

(mc,)

define SymPy ODE system

In [16]:
dp = P*(gp*ep - Rp - mp)
dm = P*(gp*(1-ep) + mp) + C*(gc*(1-ec)*M + mc)
dc = C*(gc*ec*M - Rc - mc)
dp, dm, dc

(P*(-Rp + ep*gp - mp), C*(M*gc*(1 - ec) + mc) + P*(gp*(1 - ep) + mp), C*(M*ec*gc - Rc - mc))

define eqm state

In [18]:
p_eqm = Eq(dp, 0)
m_eqm = Eq(dm, 0)
c_eqm = Eq(dc, 0)
p_eqm, m_eqm, c_eqm

(Eq(P*(-Rp + ep*gp - mp), 0), Eq(C*(M*gc*(1 - ec) + mc) + P*(gp*(1 - ep) + mp), 0), Eq(C*(M*ec*gc - Rc - mc), 0))